Notebook de ejemplo de consumo de datos del portal de transparencia del Instituto de Acceso a la Información Pública de El Salvador (https://www.transparencia.gob.sv/api/v1/home). En el ejemplo, obtenemos los datos de las instituciones registradas en el portal para con ello calcular la cantidad de días que, en promedio, cada conjunto de instituciones tiene de no actualizar sus datos o documentos.

In [ ]:
import requests
from urllib.parse import urlencode
import pandas as pd
from datetime import datetime

Primero, definimos la url base, creamos un diccionario con los parámetros a enviar al API y luego creamos la url de consulta final. Estos parámetros los obtenemos de: https://www.transparencia.gob.sv/api/v1/institutions

In [ ]:
url_base = 'https://www.transparencia.gob.sv/api/v1/institutions.json'
datos = {"start_at": "2010-01-01", "end_at":"2022-07-30", "page":1, "per_page":"1000"} #Obtiene la lista de todas las entidades
url = url_base + '?' + urlencode(datos)

Luego, utiliando la librería requests haremos una petición get y convertiremos el resultado a un diccionario de Python con la función json. Si esta falla, señalará un error; caso contrario, creará un dataframe de pandas para trabajar con él.

In [ ]:
try:
    datos = requests.get(url).json()
except:
    print("Hubo un error obteniendo los datos. Vuelve a probar.")
else:
    df = pd.DataFrame(datos)

Luego, utilizando pandas y la librería datetime se obtendrán las fechas de última actualización de las instituciones del portal de transparencia y, con ello, se calculará la cantidad de días sin actualizar.

In [ ]:
hoy = datetime.today()
df['dias'] = df['updated_at'].apply(lambda x: (hoy - datetime.strptime(x[:10], '%Y-%m-%d')).days)
print(df['dias'])

Vamos por partes. ¿Qué hace cada cosa? strptime se encarga de convertir una cadena de texto en un objeto datetime según el formato dado en el segundo parámetro de la función. Para este caso, obtenemos los 10 primeros elementos de la columna 'updated_at' (que es lo que nos interesa), y lo convertimos. Luego, al objeto datetime con la fecha de hoy le restamos la fecha obtenida mediante el API para calcular la cantidad de días sin actualizar.

In [ ]:
#A continuación calculamos el promedio de días sin actualizar para cada tipo de institución
df.groupby("institution_type_id")['dias'].mean()

In [ ]:
#Finalmente, graficamos el resultado
grafica = df.groupby("institution_type_id")['dias'].mean().plot.bar()

Por último, guardaremos los resultados de nuestros cálculos y los datos obtenidos del API en un archivo CSV para reutilizarlo cuando querramos.

In [ ]:
df.to_csv('archivo_procesado.csv')